<a href="https://colab.research.google.com/github/microize/Hackathons/blob/main/av_jobathon_data_eng_ecommerce_jul_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Problem

## Problem statement  

Your Client ComZ is an ecommerce company. The company wants to focus on targeting the right customers  with the right products to increase overall revenue and conversion rate.

To target the right customers with the right products, they need to build an ML model for marketing based on user interaction with products in the past like number of views,  most viewed product, number of activities of user, vintage of user and others. 

ComZ has contacted the Data Science and Engineering team to use this information to fuel the personalized advertisements, email marketing campaigns, or special offers on the landing and category pages of the company's website.

You, being a part of the data engineering team, are expected to “Develop input features”  for the efficient marketing model given the Visitor log data and User Data.

## Task
Need to Create ETL Pipeline 

## Link

Hackathon Link : 
https://datahack.analyticsvidhya.com/contest/job-a-thon-june-2021/#ProblemStatement

# Data

In [ ]:
# importing required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from datetime import date
from datetime import timedelta

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# importing data
visitor_logs = pd.read_csv('/content/drive/MyDrive/Data-colab/av_jobathon_data_engineer_ecommerce_etl_4_jul_21/VisitorLogsData.csv')
user_table = pd.read_csv('/content/drive/MyDrive/Data-colab/av_jobathon_data_engineer_ecommerce_etl_4_jul_21/userTable.csv')
sample_submission =pd.read_csv('/content/drive/MyDrive/Data-colab/av_jobathon_data_engineer_ecommerce_etl_4_jul_21/sample_submission_M7Vpb9f.csv')

## Data Overview

### **Visitor Log Data** – It is a browsing log data of all the visitors and the users. This table contains the following information:

* WebClientID - Unique ID of browser for every system. (If a visitor is using multiple browsers on a system like Chrome, Safari, then there would be a different web clientid for each browser). The ID remains consistent unless the user clears their cookie.

* VisitDateTime - Date and time of visit. There are two different formats for DateTime.( One is in datetime format “2018-05-07 04:28:45.970”
Another one is in unix datetime format “1527051855673000000”)

* ProductID - Unique ID of product browsed/ clicked by the visitor

* UserID - Unique ID of the registered user. As expected, this is available for registered users only, not for all visitors. 

* Activity - 
Type of activity can be browsing (pageload) or clicking a product

* Browser - 
Browser used by the visitor

* OS - Operating System of the system used by the visitor

* City - City of the visitor

* Country - Country of the visitor



### User Data – It has registered user information like signup date and segment.

* UserID - 
Unique ID of the registered user.

* Signup Date - 
Date of registration for the user

* User Segment -
User Segment (A/B/C) created based on historical engagement

Now based on the above two tables, you need to create an input feature set for the Marketing Model.


### Input Feature table:

* UserID -
Unique ID of the registered user

* No_of_days_Visited_7_Days -
How many days a user was active on platform in the last 7 days.

* No_Of_Products_Viewed_15_Days -
Number of Products viewed by the user in the last 15 days

* User_Vintage -
Vintage (In Days) of the user as of today

* Most_Viewed_product_15_Days -
Most frequently viewed (page loads) product by the user in the last 15 days. If there are multiple products that have a similar number of page loads then , consider the recent one. If a user has not viewed any product in the last 15 days then put it as Product101. 

* Most_Active_OS -
Most Frequently used OS by user. 

* Recently_Viewed_Product -
Most recently viewed (page loads) product by the user. ( If a user has not viewed any product then put it as Product101. )

* Pageloads_last_7_days -
Count of Page loads in the last 7 days by the user

* Clicks_last_7_days -
Count of Clicks in the last 7 days  by the user

## Process to create Input Feature:

When ComZ does a targeting campaign, It follows the below process. 



In the current case, you are supposed to generate an input feature set as on 28-May-2018. So, the visitor table is from 07-May-2018 to 27-May-2018.

As a Data Engineer Creating ETL Pipeline would definitely be appreciated and provide you the added advantage in interviews, Your effort should be to build ETL Pipeline such that passing the information of user data and log data, It can generate the input feature table automatically

## Example Data Points

In [ ]:
visitor_logs.head(2)

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan


In [ ]:
user_table.head(2)

,UserID,Signup Date,User Segment
0,U133159,2018-04-14 07:01:16.202607+00:00,C
1,U129368,2017-12-02 09:38:41.584270+00:00,B


In [ ]:
sample_submission.head(2)

,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,1,1,1,P12345,OS12345,P12345,1,1
1,U100003,1,1,1,P12345,OS12345,P12345,1,1


# EDA

In [ ]:
visitor_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6588000 entries, 0 to 6587999
Data columns (total 9 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   webClientID    object
 1   VisitDateTime  object
 2   ProductID      object
 3   UserID         object
 4   Activity       object
 5   Browser        object
 6   OS             object
 7   City           object
 8   Country        object
dtypes: object(9)
memory usage: 452.4+ MB


In [ ]:
user_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34050 entries, 0 to 34049
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   UserID        34050 non-null  object
 1   Signup Date   34050 non-null  object
 2   User Segment  34050 non-null  object
dtypes: object(3)
memory usage: 798.2+ KB


In [ ]:
visitor_logs.duplicated().sum()

560745

In [ ]:
user_table.duplicated().sum()

0

In [ ]:
print("Visitor_logs:",visitor_logs.shape)
print("user_table:",user_table.shape)

Visitor_logs: (6588000, 9)
user_table: (34050, 3)


## EDA - observations
* Visitor_logs shape 6588000 rows.
* User_table shape 34050 rows.
* In Visitor_logs & User_table has incorrect datatypes.
* "VisitDateTime" has mixed datetime format “2018-05-07 04:28:45.970”   & “1527051855673000000” - Need to convert unix time stamp to datetime format
* 560745 - duplicate rows are in Visitors_logs - Need to drop them
* Our campign is only for registered user. So we need to drop rows without "UserID"

# ETL Pipeline

In [ ]:
def ETL_pipeline(visit,user,a,b,c):

  final = sample_submission.copy()

  """Most_Active_OS"""

  os_df = visit.groupby(['UserID','OS'])['webClientID'].count().unstack()
  os_df['most_used'] = os_df.idxmax(axis=1)
  os_df=os_df['most_used']
  last_15_day_max_os = pd.DataFrame(os_df.reset_index())
  final = final.merge(last_15_day_max_os,how='left',left_on='UserID',right_on='UserID')

  # Dropping rows without UserID
  visit.dropna(subset=['UserID'],inplace=True)

  # Dropping duplicates
  visit.drop_duplicates(inplace=True)

  # filling missing value 
  visit['VisitDateTime']=visit['VisitDateTime'].fillna(datetime(a,b,c))

  # converting string to uppercase - to avoid inconsistency
  visit['ProductID']=visit['ProductID'].str.upper()
  visit['UserID']=visit['UserID'].str.upper()
  visit['Activity']=visit['Activity'].str.upper()
  visit['Browser']=visit['Browser'].str.upper()
  visit['OS']=visit['OS'].str.upper()
  visit['City']=visit['City'].str.upper()
  visit['Country']=visit['Country'].str.upper()

  # separating Unix timestamp
  visit['UnixTimeStamp'] = visit['VisitDateTime'].str.find(':')
  visit['UnixTimeStamp'] = visit['UnixTimeStamp'].apply(lambda x: 1 if x<=0 else 0)
  unix_visitor_logs = visit[visit['UnixTimeStamp']==1]
  norm_visitor_logs = visit[visit['UnixTimeStamp']==0]

  # converting Unix time stamp to datetime
  unix_visitor_logs['VisitDateTime'] = unix_visitor_logs['VisitDateTime'].apply(lambda x : x[:13])
  unix_visitor_logs['VisitDateTime'] = pd.to_datetime(unix_visitor_logs['VisitDateTime'],unit='ms')

  norm_visitor_logs['VisitDateTime'] = pd.to_datetime(norm_visitor_logs['VisitDateTime'])

  visit = pd.concat([unix_visitor_logs,norm_visitor_logs])

  #visitor_logs['ProductID'] = visitor_logs['ProductID'].fillna('Product101')
  visit['Activity'] = visit['Activity'].fillna('UNKNOWN')
  visit = visit.drop(['UnixTimeStamp'],axis=1)

  visit['Date'] = visit['VisitDateTime'].dt.date

  # creating separate column for click & pageloads
  visit['click'] = visit['Activity'].apply(lambda x : 1 if (x=='CLICK') else 0)
  visit['pageload'] = visit['Activity'].apply(lambda x : 1 if (x=='PAGELOAD')|(x=='UNKNOWN') else 0)

  current_date = max(visit['VisitDateTime']).date()

  
  prev_7_date = current_date-timedelta(days=7)
  prev_15_date = current_date-timedelta(days=15)

  # 7 day dataframe
  last_7_df = visit[visit['Date']>prev_7_date]
  # 15 day dataframe
  last_15_df = visit[visit['Date']>prev_15_date]

  last_7_day_click = pd.DataFrame(last_7_df.groupby(['UserID'])['click'].sum()).reset_index()
  final = final.merge(last_7_day_click,how='left',left_on='UserID',right_on='UserID')

  """Pageloads_last_7_days"""
  last_7_day_pageload = pd.DataFrame(last_7_df.groupby(['UserID'])['pageload'].sum()).reset_index()
  final = final.merge(last_7_day_pageload,how='left',left_on='UserID',right_on='UserID')

  """No_of_days_Visited_7_Days"""	
  last_7_day_visited = pd.DataFrame(last_7_df.groupby('UserID')['Date'].nunique()).reset_index()
  final = final.merge(last_7_day_visited,how='left',left_on='UserID',right_on='UserID')

  """No_Of_Products_Viewed_15_Days"""
  last_15_day_products = pd.DataFrame(last_15_df.groupby(['UserID'])['ProductID'].nunique(dropna=True)).reset_index()
  final = final.merge(last_15_day_products,how='left',left_on='UserID',right_on='UserID')

  """User_Vintage"""
  user['Signup Date'] = pd.to_datetime(user['Signup Date'])
  user['Signup_date'] = user['Signup Date'].dt.date
  user['Signup_date']=(current_date-user['Signup_date'])

  user['Signup_date'] = user['Signup_date'].dt.total_seconds()/86400
  final = final.merge(user,how='left',left_on='UserID',right_on='UserID')

  """Most_Viewed_product_15_Days"""
  product_df = last_15_df.groupby(['UserID','ProductID'])['pageload'].count().unstack()
  product_df['max_product']=product_df.idxmax(axis=1)
  product_df=product_df['max_product']
  last_15_day_max_products = pd.DataFrame(product_df.reset_index())
  final = final.merge(last_15_day_max_products,how='left',left_on='UserID',right_on='UserID')


  """Recently_Viewed_Product"""
  page_load_visitor_logs=visit[visit['pageload']==1]
  #page_load_visitor_logs['ProductID']=page_load_visitor_logs['ProductID'].fillna('PRODUCT101')
  page_load_visitor_logs.sort_values(by=['VisitDateTime'],inplace=True)
  product_recent=page_load_visitor_logs.loc[page_load_visitor_logs.groupby('UserID').VisitDateTime.idxmax()]
  product_recent=product_recent[['UserID','ProductID']]
  final = final.merge(product_recent,how='left',left_on='UserID',right_on='UserID')

  final['No_of_days_Visited_7_Days']=final['Date']
  final['No_Of_Products_Viewed_15_Days']=final['ProductID_x']
  final['User_Vintage']=final['Signup_date'].astype('int')
  final['Most_Viewed_product_15_Days']=final['max_product']
  final['Most_Active_OS']=final['most_used']
  final['Recently_Viewed_Product']=final['ProductID_y']
  final['Pageloads_last_7_days']=final['pageload']
  final['Clicks_last_7_days']=final['click']

  final = final.drop(['click', 'pageload', 'Date', 'ProductID_x','Signup Date', 'User Segment', 'Signup_date', 'max_product','most_used', 'ProductID_y'],axis=1)

  final = final.set_index('UserID')

  #visit['OS']=visit['OS'].map({'ANDROID':'Android', 'WINDOWS':'Windows', 'MAC OS X':'Mac OS X', 'LINUX':'Linux', 'UBUNTU':'Ubuntu', 'IOS':'iOS','CHROME OS':'Chrome OS', 'FEDORA':'Fedora', 'TIZEN':'Tizen'})

  final['Most_Viewed_product_15_Days']=final['Most_Viewed_product_15_Days'].fillna('Product101')
  final['Recently_Viewed_Product']=final['Recently_Viewed_Product'].fillna('Product101')

  final['Most_Viewed_product_15_Days']=final['Most_Viewed_product_15_Days'].str.upper()
  final['Recently_Viewed_Product']=final['Recently_Viewed_Product'].str.upper()

  final.fillna(0,inplace=True)

  final.to_csv('output.csv')
  return final

In [ ]:
a,b,c = 2018,5,7 # minimum date in dataframe
df = ETL_pipeline(visitor_logs,user_table,a,b,c)

# Output file

In [ ]:
df.head()

,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
UserID,,,,,,,,
U100002,0.0,2.0,52,PR100258,Android,PR100258,0.0,0.0
U100003,1.0,2.0,1020,PR100079,Windows,PR100079,1.0,2.0
U100004,1.0,15.0,340,PR100355,Windows,PRODUCT101,1.0,0.0
U100005,1.0,3.0,680,PR100234,Android,PR100234,2.0,0.0
U100006,1.0,1.0,54,PR101111,Android,PR101111,1.0,0.0
